In [1]:
import pandas as pd
import numpy as np
import random

import optuna
from optuna.samplers import TPESampler
from optuna.pruners import HyperbandPruner

import rdkit
from rdkit.Chem import AllChem, rdFingerprintGenerator, MACCSkeys
from rdkit import Chem, DataStructs

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

import matplotlib.pyplot as plt

import xgboost as xgb

c:\Users\BSH\.virtualenvs\신약개발-mImSxWNo\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOM_SEED = 999

In [5]:
def create_model(param):
  return xgb.XGBRegressor(
    random_state=RANDOM_SEED,
    tree_method="gpu_hist",
    eval_metric="rmse",
    early_stopping_rounds=50,
    **param
  )

In [ ]:
def feature_engineer():
  train_df = pd.read_csv("../data/train.csv")
  test_df = pd.read_csv("../data/test.csv")
  # 결측치 수정
  train_df = train_df.dropna()
  test_df["AlogP"] = np.where(pd.isna(test_df["AlogP"]), test_df["LogD"], test_df["AlogP"])
  
  train_df["mol"] = train_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  test_df["mol"] = test_df["SMILES"].apply(lambda x: Chem.MolFromSmiles(x))
  
  train_df['mol'] = train_df['mol'].apply(lambda x: Chem.AddHs(x))
  train_df['num_of_atoms'] = train_df['mol'].apply(lambda x: x.GetNumAtoms())
  train_df['num_of_heavy_atoms'] = train_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  test_df['mol'] = test_df['mol'].apply(lambda x: Chem.AddHs(x))
  test_df['num_of_atoms'] = test_df['mol'].apply(lambda x: x.GetNumAtoms())
  test_df['num_of_heavy_atoms'] = test_df['mol'].apply(lambda x: x.GetNumHeavyAtoms())
  
  # 중복제거
  def canonize(mol):
    return Chem.MolToSmiles(Chem.MolFromSmiles(mol), isomericSmiles=True, canonical=True)

  canon_smile = []
  for molecule in train_df['SMILES']:
      canon_smile.append(canonize(molecule))
  
  train_df['canon_smiles'] = canon_smile
  
  ind = train_df.index[train_df['canon_smiles'].duplicated()]
  train_df = train_df.drop(ind)
  
  # MLM, HLM 100 이상 제거 코드 추가 예정
  
  train_df.drop(columns=["id", "SMILES"], inplace=True)
  test_df.drop(columns=["id", "SMILES"], inplace=True)
  
  
  fmgen = rdFingerprintGenerator.GetMorganGenerator()
  
  train_fps = train_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  train_fps = np.stack(train_fps)
  
  train_maccs = train_df["mol"].apply(lambda x: list(MACCSkeys.GenMACCSKeys(x).ToBitString()))
  train_maccs = np.stack(train_maccs, dtype=object)
  
  test_fps = test_df["mol"].apply(lambda x: fmgen.GetFingerprintAsNumPy(x))
  test_fps = np.stack(test_fps)
  
  test_maccs = test_df["mol"].apply(lambda x: list(MACCSkeys.GenMACCSKeys(x).ToBitString()))
  test_maccs = np.stack(test_maccs, dtype=object)
  
  origin_train_features = train_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  origin_test_features = test_df[["AlogP", "Molecular_Weight", "Num_H_Acceptors", "Num_H_Donors", "Num_RotatableBonds", "LogD", "Molecular_PolarSurfaceArea", "num_of_atoms", "num_of_heavy_atoms"]].values
  
  train_features = np.append(np.append(origin_train_features, train_fps, axis=1), train_maccs, axis=1)
  test_features = np.append(np.append(origin_test_features, test_fps, axis=1), test_maccs, axis=1)
  target = train_df["HLM"].values
  
  return train_features, target, test_features
  

In [4]:
train_x, train_y, test_x = feature_engineer()

In [6]:
def train_model(param, X, y, X_test, trial=None):
    skf = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)

    val_scores = []
    y_tests = []
    models = []

    for idx, (train_idx, val_idx) in enumerate(skf.split(X, y)):

        X_train, y_train = X[train_idx], y[train_idx]
        X_val, y_val = X[val_idx], y[val_idx]

        model = create_model(param)
        model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_val, y_val)], verbose=1000)

        y_hat_val = model.predict(X_val)
        score = mean_squared_error(y_val, y_hat_val, squared=True)
        val_scores.append(score)
        print(f'Fold: {idx+1}/5 score = {score:.5f}')

        y_tests.append(model.predict(X_test))
        models.append(model)

        if trial:
            trial.report(score, idx)

            if trial.should_prune():
                raise optuna.TrialPruned()

    return val_scores, y_tests, models

In [7]:
def objective_xgb(trial):
    param = {
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1.0),
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
        "max_depth": trial.suggest_int("max_depth", 3, 9),
        "min_child_weight": trial.suggest_int("min_child_weight", 2, 10),
        "eta": trial.suggest_float("eta", 1e-8, 1.0, log=True),
        "gamma": trial.suggest_float("gamma", 1e-8, 1.0, log=True),
        "grow_policy": trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])
    }

    if param["booster"] == "dart":
        param['sample_type'] = trial.suggest_categorical('sample_type', ['uniform', 'weighted'])
        param['normalize_type'] = trial.suggest_categorical('normalize_type', ['tree', 'forest'])
        param['rate_drop'] = trial.suggest_float('rate_drop', 1e-8, 1.0, log=True)
        param['skip_drop'] = trial.suggest_float('skip_drop', 1e-8, 1.0, log=True)

    val_scores, y_tests, models = train_model(param, train_x, train_y, test_x, trial)

    return sum(val_scores) / len(val_scores)

In [8]:
study = optuna.create_study(
    sampler=TPESampler(seed=RANDOM_SEED),
    direction='minimize',
    study_name='xgb_tuning',
    pruner=HyperbandPruner(
        min_resource=1, max_resource=8, reduction_factor=3
    ),
)

study.optimize(objective_xgb, n_trials=100)

[I 2023-08-31 22:46:02,294] A new study created in memory with name: xgb_tuning


[0]	validation_0-rmse:63.61827	validation_1-rmse:63.84710
[499]	validation_0-rmse:31.22052	validation_1-rmse:33.58610
Fold: 1/5 score = 1128.02624
[0]	validation_0-rmse:63.87043	validation_1-rmse:62.81111
[499]	validation_0-rmse:31.36714	validation_1-rmse:32.76503
Fold: 2/5 score = 1073.54732
[0]	validation_0-rmse:63.51620	validation_1-rmse:64.27481
[499]	validation_0-rmse:31.23230	validation_1-rmse:33.78512
Fold: 3/5 score = 1141.43462
[0]	validation_0-rmse:63.49981	validation_1-rmse:64.29958
[499]	validation_0-rmse:31.30504	validation_1-rmse:33.05501
Fold: 4/5 score = 1092.63378
[0]	validation_0-rmse:63.80953	validation_1-rmse:63.08478
[499]	validation_0-rmse:31.39291	validation_1-rmse:32.44681


[I 2023-08-31 22:46:27,932] Trial 0 finished with value: 1097.6875297159982 and parameters: {'booster': 'gbtree', 'lambda': 8.972050723983975e-08, 'alpha': 0.0013105438415021338, 'subsample': 0.13637899963537597, 'colsample_bytree': 0.36561439675336166, 'n_estimators': 500, 'max_depth': 6, 'min_child_weight': 7, 'eta': 0.0037944038609466855, 'gamma': 0.02087360741176864, 'grow_policy': 'lossguide'}. Best is trial 0 with value: 1097.6875297159982.


Fold: 5/5 score = 1052.79569
[0]	validation_0-rmse:63.79702	validation_1-rmse:64.01726
[499]	validation_0-rmse:63.79558	validation_1-rmse:64.01586
Fold: 1/5 score = 4098.03054
[0]	validation_0-rmse:64.05330	validation_1-rmse:62.98567
[499]	validation_0-rmse:64.05186	validation_1-rmse:62.98429
Fold: 2/5 score = 3967.02021
[0]	validation_0-rmse:63.68925	validation_1-rmse:64.44491
[499]	validation_0-rmse:63.68782	validation_1-rmse:64.44350
Fold: 3/5 score = 4152.96418
[0]	validation_0-rmse:63.68050	validation_1-rmse:64.47949
[499]	validation_0-rmse:63.67907	validation_1-rmse:64.47807
Fold: 4/5 score = 4157.42146
[0]	validation_0-rmse:63.98468	validation_1-rmse:63.26291
[499]	validation_0-rmse:63.98324	validation_1-rmse:63.26147
Fold: 5/5 score = 4002.01423


[I 2023-08-31 23:02:02,182] Trial 1 finished with value: 4075.4901236091473 and parameters: {'booster': 'dart', 'lambda': 1.8500953639202822e-08, 'alpha': 0.18793183264405056, 'subsample': 0.43490263163831816, 'colsample_bytree': 0.7724136968223454, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 8, 'eta': 5.98025286065864e-08, 'gamma': 1.990042294039164e-05, 'grow_policy': 'lossguide', 'sample_type': 'uniform', 'normalize_type': 'tree', 'rate_drop': 4.545948976815658e-06, 'skip_drop': 0.6111537885501571}. Best is trial 0 with value: 1097.6875297159982.


...

Fold: 5/5 score = 960.87680
[0]	validation_0-rmse:63.40523	validation_1-rmse:63.63947
[599]	validation_0-rmse:23.45440	validation_1-rmse:31.83331
Fold: 1/5 score = 1012.80693
[0]	validation_0-rmse:63.67224	validation_1-rmse:62.62104
[599]	validation_0-rmse:23.56315	validation_1-rmse:31.05668
Fold: 2/5 score = 964.34822
[0]	validation_0-rmse:63.30626	validation_1-rmse:64.07084
[599]	validation_0-rmse:23.52066	validation_1-rmse:32.01228
Fold: 3/5 score = 1024.78044
[0]	validation_0-rmse:63.28887	validation_1-rmse:64.11472
[599]	validation_0-rmse:23.53599	validation_1-rmse:30.78796


[I 2023-09-01 09:05:02,822] Trial 98 pruned. 


Fold: 4/5 score = 947.89865
[0]	validation_0-rmse:62.59802	validation_1-rmse:62.90124
[323]	validation_0-rmse:20.22965	validation_1-rmse:31.91719
Fold: 1/5 score = 1015.25333
[0]	validation_0-rmse:62.85187	validation_1-rmse:61.84914
[251]	validation_0-rmse:21.48039	validation_1-rmse:30.98253
Fold: 2/5 score = 958.55456
[0]	validation_0-rmse:62.51167	validation_1-rmse:63.31077
[274]	validation_0-rmse:21.14614	validation_1-rmse:31.95900
Fold: 3/5 score = 1021.07247
[0]	validation_0-rmse:62.49717	validation_1-rmse:63.33233
[314]	validation_0-rmse:20.39717	validation_1-rmse:30.98527
Fold: 4/5 score = 956.67894
[0]	validation_0-rmse:62.81341	validation_1-rmse:62.10223
[203]	validation_0-rmse:22.39387	validation_1-rmse:31.02541


[I 2023-09-01 09:05:24,657] Trial 99 finished with value: 981.8861432280189 and parameters: {'booster': 'gbtree', 'lambda': 0.0002581879649280224, 'alpha': 0.010460479819974144, 'subsample': 0.579515446829102, 'colsample_bytree': 0.4222441323783493, 'n_estimators': 500, 'max_depth': 8, 'min_child_weight': 8, 'eta': 0.02396202036881423, 'gamma': 0.03663420935757014, 'grow_policy': 'lossguide'}. Best is trial 85 with value: 975.4777243045952.


Fold: 5/5 score = 957.87141


In [9]:
trial = study.best_trial

In [10]:
print(trial.params)

{'booster': 'gbtree', 'lambda': 0.0004765481568976222, 'alpha': 0.12415583410758738, 'subsample': 0.5632111334907796, 'colsample_bytree': 0.5172928782547614, 'n_estimators': 700, 'max_depth': 8, 'min_child_weight': 8, 'eta': 0.0068054260738828485, 'gamma': 0.028544481456365057, 'grow_policy': 'depthwise'}


In [11]:
best_param = {
  'booster': 'gbtree',
  'lambda': 0.0004765481568976222,
  'alpha': 0.12415583410758738,
  'subsample': 0.5632111334907796,
  'colsample_bytree': 0.5172928782547614,
  'n_estimators': 700,
  'max_depth': 8,
  'min_child_weight': 8,
  'eta': 0.0068054260738828485,
  'gamma': 0.028544481456365057,
  'grow_policy': 'depthwise'
}

val_scores, y_tests, models = train_model(best_param, train_x, train_y, test_x)

[0]	validation_0-rmse:63.45338	validation_1-rmse:63.69175
[699]	validation_0-rmse:22.42570	validation_1-rmse:31.83782
Fold: 1/5 score = 1012.52503
[0]	validation_0-rmse:63.71043	validation_1-rmse:62.66427
[699]	validation_0-rmse:22.56443	validation_1-rmse:30.82465
Fold: 2/5 score = 950.15910
[0]	validation_0-rmse:63.35345	validation_1-rmse:64.12165
[699]	validation_0-rmse:22.38911	validation_1-rmse:31.79765
Fold: 3/5 score = 1010.57593
[0]	validation_0-rmse:63.34294	validation_1-rmse:64.14646
[699]	validation_0-rmse:22.41343	validation_1-rmse:30.72075
Fold: 4/5 score = 943.76466
[0]	validation_0-rmse:63.64251	validation_1-rmse:62.93098
[699]	validation_0-rmse:22.46194	validation_1-rmse:31.00025
Fold: 5/5 score = 960.36390


In [13]:
len(models)

5

In [14]:
for idx, model in enumerate(models):
  model.save_model(f"xgboost_HLM_fold_{idx}.model")

In [ ]:
# importance = models[-1].get_booster().get_score(importance_type="weight")
# tuples = [(k, importance[k]) for k in importance]
# tuples.sort(key=lambda x: x[1])

# label, values = zip(*tuples)

# plt.rcParams["font.family"] = "MalGun Gothic"
# plt.figure(figsize=(10, 12))
# plt.barh(label[:100], values[:100])
# plt.show()

In [24]:
submit = pd.read_csv("../data/sample_submission.csv")
submit["HLM"] = np.mean(y_tests, axis=0)
submit.to_csv("xgboost_optuna_v1.csv", index=False)